In [15]:
import pandas as pd

In [16]:
log_path = r'D:\Shilong\new_murmur\01_code\AutoMTL\data_format1\user_log_format1.csv'
info_path = r'D:\Shilong\new_murmur\01_code\AutoMTL\data_format1\user_info_format1.csv'

In [17]:
log_df = pd.read_csv(log_path)

In [18]:
log_df.dropna(inplace=True)
log_df['brand_id'] = log_df['brand_id'].astype('int')
log_df = log_df.groupby(by=['user_id', 'item_id', 'cat_id', 'seller_id', 'brand_id'], as_index=False)['action_type'].agg(list)

In [19]:
log_df['purchase'] = log_df['action_type'].map(lambda x: int(2 in x))
log_df['favourite'] = log_df['action_type'].map(lambda x: int(3 in x))
log_df.drop('action_type', axis=1, inplace=True)

In [20]:
user_df = pd.read_csv(info_path)
user_df.dropna(inplace=True)
user_df['age_range'] = user_df['age_range'].astype('int')
user_df['gender'] = user_df['gender'].astype('int')

In [21]:
data_df = pd.merge(log_df, user_df, how='left', on='user_id')

In [22]:
data_df.dropna(inplace=True)

In [23]:
data_df

,user_id,item_id,cat_id,seller_id,brand_id,purchase,favourite,age_range,gender
0,1,112203,1252,4026,1469,1,0,3.0,1.0
1,1,181459,276,2245,4750,0,0,3.0,1.0
2,1,411984,1252,4177,1960,0,0,3.0,1.0
3,1,452339,389,4335,649,0,0,3.0,1.0
4,1,452837,276,2245,4750,0,0,3.0,1.0
...,...,...,...,...,...,...,...,...,...
32010134,424170,678194,1397,4963,1991,0,0,0.0,1.0
32010135,424170,701110,602,525,5444,0,0,0.0,1.0
32010136,424170,795753,656,4268,1642,1,0,0.0,1.0
32010137,424170,802762,602,3736,3125,0,0,0.0,1.0


In [24]:
def filte(df: pd.DataFrame, field_name, target_field='purchase', thresh=5):
    items = set()
    df = df.groupby(by=field_name, as_index=False)[target_field].agg(sum)
    for k, v in df.values:
        if v < thresh:
            items.add(k)

    return items

In [25]:
ban_users = filte(data_df, 'user_id', thresh=10)
ban_items = filte(data_df, 'item_id', thresh=10)

In [26]:
drop_user = data_df['user_id'].isin(ban_users)
data_df.drop(data_df.index[drop_user], inplace=True)

In [27]:
drop_item = data_df['item_id'].isin(ban_items)
data_df.drop(data_df.index[drop_item], inplace=True)

In [28]:
fields = {
    'user_id': 'user_id:token',
    'item_id': 'item_id:token',
    'cat_id': 'cat_id:token',
    'seller_id': 'seller_id:token',
    'brand_id': 'brand_id:token',
    'age_range': 'age_range:token',
    'gender': 'gender:token',
    'purchase': 'purchase:label',
    'favourite': 'favourite:label'
}
data_df.rename(columns=fields, inplace=True)

In [29]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
lbe = LabelEncoder()
data_df["user_id:token"] = lbe.fit_transform(data_df["user_id:token"]) + 1
data_df["item_id:token"] = lbe.fit_transform(data_df["item_id:token"]) + 1
data_df["cat_id:token"] = lbe.fit_transform(data_df["cat_id:token"]) + 1
data_df["seller_id:token"] = lbe.fit_transform(data_df["seller_id:token"]) + 1
data_df["brand_id:token"] = lbe.fit_transform(data_df["brand_id:token"]) + 1

In [30]:
df_train, df_val = train_test_split(data_df, test_size=0.2)
df_val, df_test = train_test_split(df_val, test_size=0.5)

df_train.to_csv("ijcai15_train.csv", index=False)
df_val.to_csv("ijcai15_val.csv", index=False)
df_test.to_csv("ijcai15_test.csv", index=False)

In [31]:
data_df.to_csv('ijcai_15.inter', index=False)